In [1]:
import tensorflow as tf
import math
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, PReLU, Flatten, Softmax, ReLU
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.models import load_model
from tools.data_handling import get_image_paths, sample_data, load_data
import numpy as np
import os

In [2]:
def PNet():
        
    X = Input(shape = (12, 12, 3), name='PNet_Input')
    
    L = Conv2D(10, kernel_size=(3, 3), strides=(1, 1), padding='valid', name='PNet_CONV1')(X)
    L = ReLU(name='PNet_RELU1')(L)
    L = MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='same', name='PNet_MAXPOOL1')(L)

    L = Conv2D(16, kernel_size=(3, 3), strides=(1, 1), padding='valid', name='PNet_CONV2')(L)
    L = ReLU(name='PNet_RELU2')(L)

    L = Conv2D(32, kernel_size=(3, 3), strides=(1, 1), padding='valid', name='PNet_CONV3')(L)
    L = ReLU(name='PNet_RELU3')(L)

    C = Conv2D(2, kernel_size=(1, 1), strides=(1, 1), name = 'PNet_CONV4')(L)
    classifier = Softmax(name='FACE_CLASSIFIER')(C)
    regressor = Conv2D(4, kernel_size=(1, 1), strides=(1, 1), name = 'BB_REGRESSION')(L)

    return Model(X, [regressor, classifier], name = 'PNet')

In [3]:
def RNet():

    X = Input(shape = (24,24,3), name='RNet_Input')

    L = Conv2D(28, kernel_size=(3, 3), strides=(1, 1), padding='valid', name='RNet_CONV1')(X)
    L = ReLU(name='RNet_RELU1')(L)
    L = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same', name='RNet_MAXPOOL1')(L)

    L = Conv2D(48, kernel_size=(3, 3), strides=(1, 1), padding='valid', name='ENet_CONV2')(L)
    L = ReLU(name='RNet_RELU2')(L)
    L = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='valid', name='RNet_MAXPOOL2')(L)

    L = Conv2D(64, kernel_size=(2, 2), strides=(1, 1), padding='valid', name='RNet_CONV3')(L)
    L = ReLU(name='RNet_RELU3')(L)
    L = Flatten(name = 'RNet_FLATTEN')(L)
    L = Dense(128, name = 'RNet_DENSE1')(L)
    L = ReLU(name = 'RNet_PRELU4')(L)

    C = Dense(2, name = 'RNet_DENSE2')(L)
    classifier = Softmax(axis=1, name = 'FACE_CLASSIFIER')(C)
    regressor = Dense(4, name = 'BB_REGRESSION')(L)

    return Model(X, [regressor, classifier], name = 'RNet')


In [4]:
def ONet():
    
    X = Input(shape = (48, 48, 3), name = 'ONet_input')

    L = Conv2D(32, kernel_size= (3,3), strides = (1,1), padding = 'valid', name = 'ONet_CONV1')(X)
    L = ReLU(name = 'ONet_RELU1')(L)
    L = MaxPooling2D(pool_size = (3,3), strides = (2, 2), padding = 'same', name = 'RNet_MAXPOOL1')(L)
        
    L = Conv2D(64, kernel_size= (3,3), strides = (1,1), padding = 'valid', name = 'ONet_CONV2')(L)
    L = ReLU(name = 'ONet_RELU2')(L)
    L = MaxPooling2D(pool_size = (3,3), strides = (2, 2), padding = 'valid', name = 'RNet_MAXPOOL2')(L)
        
    L = Conv2D(64, kernel_size= (3,3), strides = (1,1), padding = 'valid', name = 'ONet_CONV3')(L)
    L = ReLU(name = 'ONet_RELU3')(L)
    L = MaxPooling2D(pool_size = (2, 2), strides=(2, 2), padding = 'same', name = 'RNet_MAXPOOL3')(L)
    
    L = Conv2D(128, kernel_size= (2,2), strides = (1, 1), padding = 'valid', name = 'ONet_CONV4')(L)
    L = ReLU(name='ONet_RELU4')(L)
    
    L = Flatten(name = 'ONet_FLATTEN')(L)
    L = Dense(256, name = 'ONet_DENSE1') (L)
    L = ReLU(name = 'ONet_RELU5')(L)

    C = Dense(2, name = 'ONet_DENSE2')(L)
    classifier = Softmax(axis = 1, name = 'FACE_CLASSIFIER')(C)
    regressor = Dense(4, name = 'BB_REGRESSION')(L)

    return Model(X, [regressor, classifier], name = 'ONet')

In [5]:
pixels = 48
model = ONet()

In [6]:
model_name = 'onet'

In [7]:
model.summary()

Model: "ONet"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
ONet_input (InputLayer)         [(None, 48, 48, 3)]  0                                            
__________________________________________________________________________________________________
ONet_CONV1 (Conv2D)             (None, 46, 46, 32)   896         ONet_input[0][0]                 
__________________________________________________________________________________________________
ONet_RELU1 (ReLU)               (None, 46, 46, 32)   0           ONet_CONV1[0][0]                 
__________________________________________________________________________________________________
RNet_MAXPOOL1 (MaxPooling2D)    (None, 23, 23, 32)   0           ONet_RELU1[0][0]                 
_______________________________________________________________________________________________

In [8]:
# get training data
training_image_paths = get_image_paths(pixels, 'raw')
t_lengths = []
for path in training_image_paths:
    (_, anno_path) = path
    anno = open(anno_path, 'r')
    t_lengths.append(len(anno.readlines()))
validation_image_paths = get_image_paths(pixels, 'val')
v_lengths = []
for path in validation_image_paths:
    (_, anno_path) = path
    anno = open(anno_path, 'r')
    v_lengths.append(len(anno.readlines()))

In [9]:
t_split, v_split = [1, 0, 1], [1, 0, 1]
training_size = 1000
validation_size = 200
t_numbers = [math.ceil(training_size / sum(t_split) * s) for s in t_split]
v_numbers = [math.ceil(validation_size / sum(v_split) * s) for s in v_split]

In [10]:
for l, n in zip(t_lengths, t_numbers):
    if l < n: logging.fatal('There are not enough samples')
for l, n in zip(v_lengths, v_numbers):
    if l < n: logging.fatal('There are not enough samples')

In [11]:
t_samples = sample_data(t_numbers, training_image_paths)
v_samples = sample_data(v_numbers, validation_image_paths)

In [12]:
t_data, t_cat, t_bbx = load_data(t_samples, pixels)
v_data, v_cat, v_bbx = load_data(v_samples, pixels)

100%|██████████| 200/200 [00:00<00:00, 1575.77it/s]


In [13]:
losses = {
    'FACE_CLASSIFIER' : tf.keras.losses.BinaryCrossentropy(),
    'BB_REGRESSION' : tf.keras.losses.MeanSquaredError()
}
loss_weights = {
    'FACE_CLASSIFIER' : 1.0,
    'BB_REGRESSION' : 0.5
}

# compile model
model.compile(
    loss = losses,
    loss_weights=loss_weights,
    optimizer = tf.keras.optimizers.Adam(),
    metrics=['accuracy', 'mse']
)

# train
H = model.fit(
    x=t_data,
    y={
        'FACE_CLASSIFIER' : t_cat,
        'BB_REGRESSION' : t_bbx
        },
    batch_size=64,
    epochs=50
)

Epoch 1/50
16/16 [==============================] - 2s 109ms/step - loss: 0.6246 - BB_REGRESSION_loss: 0.0085 - FACE_CLASSIFIER_loss: 0.6203 - BB_REGRESSION_accuracy: 0.4680 - BB_REGRESSION_mse: 0.0085 - FACE_CLASSIFIER_accuracy: 0.6390 - FACE_CLASSIFIER_mse: 0.2159
Epoch 2/50
16/16 [==============================] - 2s 98ms/step - loss: 0.4946 - BB_REGRESSION_loss: 0.0074 - FACE_CLASSIFIER_loss: 0.4909 - BB_REGRESSION_accuracy: 0.2870 - BB_REGRESSION_mse: 0.0074 - FACE_CLASSIFIER_accuracy: 0.7780 - FACE_CLASSIFIER_mse: 0.1558
Epoch 3/50
16/16 [==============================] - 2s 103ms/step - loss: 0.4436 - BB_REGRESSION_loss: 0.0065 - FACE_CLASSIFIER_loss: 0.4404 - BB_REGRESSION_accuracy: 0.4220 - BB_REGRESSION_mse: 0.0065 - FACE_CLASSIFIER_accuracy: 0.8110 - FACE_CLASSIFIER_mse: 0.1379
Epoch 4/50
16/16 [==============================] - 2s 114ms/step - loss: 0.4407 - BB_REGRESSION_loss: 0.0069 - FACE_CLASSIFIER_loss: 0.4372 - BB_REGRESSION_accuracy: 0.4090 - BB_REGRESSION_mse: 0.006

16/16 [==============================] - 2s 154ms/step - loss: 0.0420 - BB_REGRESSION_loss: 0.0055 - FACE_CLASSIFIER_loss: 0.0392 - BB_REGRESSION_accuracy: 0.4460 - BB_REGRESSION_mse: 0.0055 - FACE_CLASSIFIER_accuracy: 0.9860 - FACE_CLASSIFIER_mse: 0.0100
Epoch 32/50
16/16 [==============================] - 3s 160ms/step - loss: 0.0295 - BB_REGRESSION_loss: 0.0050 - FACE_CLASSIFIER_loss: 0.0270 - BB_REGRESSION_accuracy: 0.4080 - BB_REGRESSION_mse: 0.0050 - FACE_CLASSIFIER_accuracy: 0.9900 - FACE_CLASSIFIER_mse: 0.0072
Epoch 33/50
16/16 [==============================] - 2s 151ms/step - loss: 0.0215 - BB_REGRESSION_loss: 0.0049 - FACE_CLASSIFIER_loss: 0.0191 - BB_REGRESSION_accuracy: 0.4290 - BB_REGRESSION_mse: 0.0049 - FACE_CLASSIFIER_accuracy: 0.9960 - FACE_CLASSIFIER_mse: 0.0042
Epoch 34/50
16/16 [==============================] - 2s 149ms/step - loss: 0.0145 - BB_REGRESSION_loss: 0.0047 - FACE_CLASSIFIER_loss: 0.0122 - BB_REGRESSION_accuracy: 0.4540 - BB_REGRESSION_mse: 0.0047 - FAC

In [14]:
score = model.evaluate(
    x=v_data,
    y={
        'FACE_CLASSIFIER' : v_cat,
        'BB_REGRESSION' : v_bbx
        },
    batch_size=64
)

4/4 [==============================] - 0s 23ms/step - loss: 0.7603 - BB_REGRESSION_loss: 0.0059 - FACE_CLASSIFIER_loss: 0.7573 - BB_REGRESSION_accuracy: 0.5000 - BB_REGRESSION_mse: 0.0059 - FACE_CLASSIFIER_accuracy: 0.8800 - FACE_CLASSIFIER_mse: 0.1018


In [15]:
score

[0.7602623701095581,
 0.005854387301951647,
 0.7573351860046387,
 0.5,
 0.005854387301951647,
 0.8799999952316284,
 0.10176583379507065]

In [16]:
# convert to tf lite
converter = tf.lite.TFLiteConverter.from_keras_model(model)

In [17]:
quantize = True
if (quantize):
    def representative_dataset():
        for i in range(500):
            yield([t_data[i].reshape(1,pixels,pixels,3)])
    # Set the optimization flag.
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    # Enforce full-int8 quantization
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
    converter.inference_input_type = tf.int8  # or tf.uint8
    converter.inference_output_type = tf.int8  # or tf.uint8
    # Provide a representative dataset to ensure we quantize correctly.
converter.representative_dataset = representative_dataset

In [18]:
tflitemodel = converter.convert()

Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


INFO:tensorflow:Assets written to: /var/folders/43/5blqqb3n63j88pcfn03dqgcm0000gn/T/tmp1v211t9j/assets


INFO:tensorflow:Assets written to: /var/folders/43/5blqqb3n63j88pcfn03dqgcm0000gn/T/tmp1v211t9j/assets


In [19]:
open(os.path.join(model_name + '.tflite'), 'wb').write(tflitemodel)

402752

In [20]:
tflite_interpreter = tf.lite.Interpreter(model_path=model_name + '.tflite')
tflite_interpreter.allocate_tensors()
input_details = tflite_interpreter.get_input_details()
output_details = tflite_interpreter.get_output_details()

In [21]:
input_details

[{'name': 'ONet_input',
  'index': 0,
  'shape': array([ 1, 48, 48,  3], dtype=int32),
  'shape_signature': array([-1, 48, 48,  3], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.003921568859368563, -128),
  'quantization_parameters': {'scales': array([0.00392157], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [22]:
output_details

[{'name': 'Identity',
  'index': 25,
  'shape': array([1, 4], dtype=int32),
  'shape_signature': array([-1,  4], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.002142949728295207, -8),
  'quantization_parameters': {'scales': array([0.00214295], dtype=float32),
   'zero_points': array([-8], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}},
 {'name': 'Identity_1',
  'index': 27,
  'shape': array([1, 2], dtype=int32),
  'shape_signature': array([-1,  2], dtype=int32),
  'dtype': numpy.int8,
  'quantization': (0.00390625, -128),
  'quantization_parameters': {'scales': array([0.00390625], dtype=float32),
   'zero_points': array([-128], dtype=int32),
   'quantized_dimension': 0},
  'sparsity_parameters': {}}]

In [23]:
cat_predictions = np.zeros((len(v_data),), dtype= int)
bbx_predictions = np.zeros((len(v_data), 4), dtype= int)
input_scale, input_zero_point = input_details[0]['quantization']

In [24]:
for i in range(len(v_data)):
    val_batch = v_data[i]
    val_batch = val_batch / input_scale + input_zero_point
    val_batch = np.expand_dims(val_batch, axis = 0).astype(input_details[0]['dtype'])
    tflite_interpreter.set_tensor(input_details[0]['index'], val_batch)
    tflite_interpreter.allocate_tensors()
    tflite_interpreter.invoke()
    
    # print("Prediction results shape:", tflite_model_bbx_predictions.shape)
    cat_output = tflite_interpreter.get_tensor(output_details[1]['index'])
    bbx_output = tflite_interpreter.get_tensor(output_details[0]['index'])
    cat_predictions[i] = cat_output.argmax()
    bbx_predictions[i] = bbx_output

In [25]:
bbx_scale, bbx_zero_point = output_details[0]['quantization']
bbx_predictions_f = (bbx_predictions.astype(float) - bbx_zero_point) * bbx_scale

In [26]:
v_bbx = v_bbx.reshape(bbx_predictions_f.shape)

In [27]:
mse = np.square(v_bbx - bbx_predictions_f).mean(axis=0).mean()

In [28]:
mse

0.0058238648640499165

In [29]:
ground_truth = [i.argmax() for i in v_cat.astype(int)]

In [41]:
q_score = sum([1 for p, g in zip(cat_predictions, ground_truth) if p == g]) / len(ground_truth)

In [51]:
n_score = model.evaluate(x= v_data, y = {'FACE_CLASSIFIER' : v_cat, 'BB_REGRESSION' : v_bbx})

7/7 [==============================] - 0s 13ms/step - loss: 0.7603 - BB_REGRESSION_loss: 0.0059 - FACE_CLASSIFIER_loss: 0.7573 - BB_REGRESSION_accuracy: 0.5000 - BB_REGRESSION_mse: 0.0059 - FACE_CLASSIFIER_accuracy: 0.8800 - FACE_CLASSIFIER_mse: 0.1018


In [52]:
n_score

[0.7602624297142029,
 0.005854387301951647,
 0.7573351860046387,
 0.5,
 0.005854387301951647,
 0.8799999952316284,
 0.10176583379507065]

In [53]:
print('That is a change of %s percent' % ((n_score[5] - q_score) * 100))

That is a change of -4.768371586472142e-07 percent
